In [2]:
#libraries
import illustris_python as il
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.spatial import cKDTree
from scipy.interpolate import griddata
from scipy import interpolate

from astropy import constants as const

# Constants
G = const.G.cgs.value  # Gravitational constant in cm^3 g^-1 s^-2
m_p = const.m_p.cgs.value  # Proton mass in g
c = const.c.cgs.value  # Speed of light in cm/s
sigma_T = const.sigma_T.cgs.value  # Thomson cross-section in cm^2

e_r = 0.2 #radiative accretion efficiency

e_fh = 0.05 #high-accretion state 
e_fm = 0.2 #low-accretion state

Msun = 1.989e33 # in grams
Gyr_to_s = 3.15576e16 # in seconds

h = 0.6774 #for illustrisTNG

#Path of the simulation
basePath = './sims.TNG/TNG100-1/output/'

### Prediction data for z=1 same way with the data for illustris and illustrisTNG

In [2]:
snapshot = 50

#load the index of Subhalo table of the first (central) Subfind subhalo within this FoF group
GroupFirstSub = il.groupcat.loadHalos(basePath,snapshot,fields=['GroupFirstSub'])
print(GroupFirstSub.dtype, GroupFirstSub.shape)


#load the integer counter of the total number of particles/cells, split into the six different types, in this group.
GroupLenType = il.groupcat.loadHalos(basePath,snapshot,fields=['GroupLenType'])
print(GroupLenType.dtype, GroupLenType.shape)

int32 (6736881,)
int32 (6736881, 6)


In [3]:
# Particle type number for black holes
ptNumBH = il.snapshot.partTypeNum('bh')

# Array of number of black holes in each halo
numBH = GroupLenType[:, ptNumBH] #ptNumBH = 5

# Find indices of halos that contain exactly one black hole and at least one subhalo
w = np.where((numBH == 1) & (GroupFirstSub >= 0))[0]

# Find the first subhalo in each of these halos
galaxies = GroupFirstSub[w]

len(galaxies)

22254

In [4]:
#ids of 10 most massive galaxies
ids = galaxies[0:10000]
len(ids)

10000

In [5]:
# Convert list to pandas DataFrame
df = pd.DataFrame(ids, columns=['ID'])

# Save to CSV
df.to_csv('z=1.csv', index=False)

In [6]:
#lets view the compilation of the file 
df = pd.read_csv('z=1.csv')
print(df.head())

       ID
0  199097
1  210307
2  212195
3  222547
4  223589


In [7]:
# Create empty lists to store the fraction χ of the Eddington accretion rate values
χ_rate_list1 = []  # for χ_rate >= 1, high accretion state
χ_rate_list2 = []  # for χ_rate < 1, low accretion state

fields=["BH_Mass", "BH_Mdot", "BH_MdotEddington"]


# Loop over each subhalo ID
for id in ids:
    # Load black hole properties for specific subhalo
    blackholes = il.snapshot.loadSubhalo(basePath, snapshot, id, "BH", fields=fields)
    
    # Calculate the BHdot.
    if isinstance(blackholes, np.ndarray) and blackholes.size > 0 and blackholes['BH_Mdot'][0] != 0:  
        bhdot = blackholes['BH_Mdot'][0]
    elif isinstance(blackholes, dict) and 'BH_Mdot' in blackholes and len(blackholes['BH_Mdot']) > 0 and blackholes['BH_Mdot'][0] != 0: 
        bhdot = blackholes['BH_Mdot'][0]
    else:  
        bhdot = np.nan  # Set to np.nan so that the resulting χ_rate will be np.nan.
        
    # Calculate the BH eddington.
    if isinstance(blackholes, np.ndarray) and blackholes.size > 0 and blackholes['BH_MdotEddington'][0] != 0:  
        bhedd = blackholes['BH_MdotEddington'][0]
    elif isinstance(blackholes, dict) and 'BH_MdotEddington' in blackholes and len(blackholes['BH_MdotEddington']) > 0 and blackholes['BH_MdotEddington'][0] != 0:  
        bhedd = blackholes['BH_MdotEddington'][0]
    else:  
        bhedd = np.nan  # Set to np.nan so that the resulting χ_rate will be np.nan.
        
    # Calculate χ_rate
    χ_rate = (bhdot / bhedd)
        
    # Append χ_rate to the appropriate list
    if χ_rate >= 1:
        χ_rate_list1.append(χ_rate)
    else:
        χ_rate_list2.append(χ_rate)

# Print fraction χ values
print("χ values for χ_rate >= 1:")
print(len(χ_rate_list1))

print("χ values for χ_rate < 1:")
print(len(χ_rate_list2))

χ values for χ_rate >= 1:
3
χ values for χ_rate < 1:
9997


#### Parameters 

1) Energy feed 
<br>
2) Bolometric luminosity
<br>
3) SubhaloSFRinHalfRad
<br>
4) Average divergence velocity
<br>
5) SubhaloVelDisp
<br>
6) BH_Mdot
<br>
7) BH_Mass
<br>
8) SubhaloGasMetallicitySfr
<br>
9) SubhaloStellarPhotometrics

χ and λ is the same thing

In [7]:
# Create empty list to store of fraction χ of the Eddington accretion rate values
λ_rate_list = []

fields=["BH_Mass", "BH_Mdot", "BH_MdotEddington"]

# Loop over each subhalo ID and there snapshot
for id in ids:
    # Load black hole properties for specific subhalo
    blackholes = il.snapshot.loadSubhalo(basePath, snapshot, id, "BH", fields=fields)
    
    # Calculate the BHdot.
    if isinstance(blackholes, np.ndarray) and blackholes.size > 0 and blackholes['BH_Mdot'][0] != 0:  
        bhdot = blackholes['BH_Mdot'][0]
    elif isinstance(blackholes, dict) and 'BH_Mdot' in blackholes and len(blackholes['BH_Mdot']) > 0 and blackholes['BH_Mdot'][0] != 0: 
        bhdot = blackholes['BH_Mdot'][0]
    else:  
        bhdot = np.nan  # Set to np.nan so that the resulting λ_rate will be np.nan.
        
    # Calculate the BH eddington.
    if isinstance(blackholes, np.ndarray) and blackholes.size > 0 and blackholes['BH_MdotEddington'][0] != 0:  
        bhedd = blackholes['BH_MdotEddington'][0]
    elif isinstance(blackholes, dict) and 'BH_MdotEddington' in blackholes and len(blackholes['BH_MdotEddington']) > 0 and blackholes['BH_MdotEddington'][0] != 0:  
        bhedd = blackholes['BH_MdotEddington'][0]
    else:  
        bhedd = np.nan  # Set to np.nan so that the resulting λ_rate will be np.nan.

        
    # Calculate λ_rate
    λ_rate = (bhdot / bhedd)

    
    # Append λ_rate to the list
    λ_rate_list.append(λ_rate)


print("λ values:")
print(len(λ_rate_list))

df = pd.read_csv('z=1.csv')
df['λ'] = λ_rate_list
df.to_csv('z=1.csv', index=False)

λ values:
10000


In [8]:
# Create empty list to store the feedback energy of the AGN values
E_feed_list = []


Msun = 1.989e33 # in grams
Gyr_to_s = 3.15576e16 # in seconds
h = 0.6774

fields=["BH_Mass", "BH_Mdot"]

# Loop over each subhalo ID and there snapshot
for id in ids:
    # Load black hole properties for specific subhalo
    blackholes = il.snapshot.loadSubhalo(basePath, snapshot, id, "BH", fields=fields)
    
    # Calculate the BHdot.
    if isinstance(blackholes, np.ndarray) and blackholes.size > 0 and blackholes['BH_Mdot'][0] != 0:  
        bhdot = blackholes['BH_Mdot'][0] * ((1e10 * Msun) / (h * Gyr_to_s))
    elif isinstance(blackholes, dict) and 'BH_Mdot' in blackholes and len(blackholes['BH_Mdot']) > 0 and blackholes['BH_Mdot'][0] != 0: 
        bhdot = blackholes['BH_Mdot'][0] * ((1e10 * Msun) / (h * Gyr_to_s))
    else:  
        bhdot = np.nan  # Set to np.nan so that the resulting feedback energy will be np.nan.

        
    # Calculate feedback energy
    E_feed = e_fm * bhdot * c**2

    
    # Append feedback energy to the list
    E_feed_list.append(np.log10(E_feed))


# Print feedback energy values
print("E_feed values:")

print(len(E_feed_list))

df = pd.read_csv('z=1.csv')
df['E_feed'] = E_feed_list
df.to_csv('z=1.csv', index=False)

E_feed values:
10000


In [9]:
# Create empty list to store the bolometric luminosity values
L_list = []

fields=["BH_Mass", "BH_Mdot"]

# Loop over each subhalo ID and there snapshot
for id in ids:
    # Load black hole properties for specific subhalo
    blackholes = il.snapshot.loadSubhalo(basePath, snapshot, id, "BH", fields=fields)
    
    # Calculate the BHdot.
    if isinstance(blackholes, np.ndarray) and blackholes.size > 0 and blackholes['BH_Mdot'][0] != 0:  
        bhdot = blackholes['BH_Mdot'][0] * ((1e10 * Msun) / (h * Gyr_to_s))
    elif isinstance(blackholes, dict) and 'BH_Mdot' in blackholes and len(blackholes['BH_Mdot']) > 0 and blackholes['BH_Mdot'][0] != 0: 
        bhdot = blackholes['BH_Mdot'][0] * ((1e10 * Msun) / (h * Gyr_to_s))
    else:  
        bhdot = np.nan  # Set to np.nan so that the resulting L_bol will be np.nan.

    # Calculate L_bol
    L_bol = e_r * bhdot * c**2

    
    # Append bolometric luminosity to the list
    L_list.append(np.log10(L_bol))


# Print bolometric luminosity values
print("Bol Luminosity values:")
print(len(L_list))

df = pd.read_csv('z=1.csv')
df['L_bol'] = L_list
df.to_csv('z=1.csv', index=False)

Bol Luminosity values:
10000


In [10]:
# List to store Star formation rate
sfr_values = []

# Loop over each subhalo ID and there snapshot
for id in ids:
    #load the galaxies
    sfr = il.groupcat.loadSingle(basePath, snapshot, subhaloID=id)
    #extract the value of the sfr
    sfr_value = sfr['SubhaloSFRinHalfRad']
    sfr_values.append(sfr_value)
    
print("SFR values:")
print(len(sfr_values))

df = pd.read_csv('z=1.csv')
df['SFRinhalfrad'] = sfr_values
df.to_csv('z=1.csv', index=False)

SFR values:
10000


In [ ]:
def calculate_divergence(part_data, agn_position):
    # Calculate the position of the particles relative to the AGN
    relative_pos = part_data['Coordinates'] - agn_position

    # Calculate the position boundaries
    padding = 1  # This value can be adhust as needed
    min_x, min_y, min_z = np.min(relative_pos, axis=0) - padding
    max_x, max_y, max_z = np.max(relative_pos, axis=0) + padding

    # Create a grid
    grid_x, grid_y, grid_z = np.mgrid[min_x:max_x:100j, min_y:max_y:100j, min_z:max_z:100j]

    # Interpolate the velocities onto the grid
    grid_vx = interpolate.griddata(relative_pos, part_data['Velocities'][:, 0], (grid_x, grid_y, grid_z), method='nearest')
    grid_vy = interpolate.griddata(relative_pos, part_data['Velocities'][:, 1], (grid_x, grid_y, grid_z), method='nearest')
    grid_vz = interpolate.griddata(relative_pos, part_data['Velocities'][:, 2], (grid_x, grid_y, grid_z), method='nearest')

    # Calculate the divergence
    divergence = np.gradient(grid_vx, axis=0) + np.gradient(grid_vy, axis=1) + np.gradient(grid_vz, axis=2)
    
    # Return the average divergence    
    return np.mean(divergence)

# Define lists to store the results
avg_divergence_list = []

# Loop over each subhalo ID and there snapshot
for id in ids:
    # Load the particle data for the subhalo
    part_data = il.snapshot.loadSubhalo(basePath, snapshot, id, partType='gas', fields=['Coordinates', 'Velocities'])
    
    agn_position = il.groupcat.loadSingle(basePath, snapshot, subhaloID=id)
    agn_position = agn_position['SubhaloPos']

    # Calculate the divergence
    average_divergence = calculate_divergence(part_data, agn_position)

    # Store the results
    avg_divergence_list.append(average_divergence)

# Print the average divergence
print("Average divergence velocity values:")
print(len(avg_divergence_list))

#read the file    
df = pd.read_csv('z=1.csv')
#append them to the new column 
df['Average divergence'] = avg_divergence_list
df.to_csv('z=1.csv', index=False)

In [5]:
def calculate_radial_velocity(positions, velocities, agn_position):
    # Calculate the relative positions and distances of the particles
    relative_positions = positions - agn_position
    distances = np.linalg.norm(relative_positions, axis=1)

    # Calculate the radial velocities
    radial_velocities = np.sum(relative_positions * velocities, axis=1) / distances

    return radial_velocities

# Define lists to store the results
average_radial_velocity_list = []


# Loop over each subhalo ID and there snapshot
for id in ids:
    # Get the main halo ID (Group Number) for the subhalo
    main_halo_id = il.groupcat.loadSingle(basePath, snapshot, subhaloID=id)['SubhaloGrNr']
    
    # Load the properties of the main halo
    main_halo_properties = il.groupcat.loadSingle(basePath, snapshot, haloID=main_halo_id)
    
    # Get the position of the main halo
    agn_position = main_halo_properties['GroupPos']
    
    # Load the particle data for the halo
    part_data = il.snapshot.loadHalo(basePath, snapshot, main_halo_id, partType='gas', fields=['Coordinates', 'Velocities'])

    # Calculate the radial velocities
    radial_velocities = calculate_radial_velocity(part_data['Coordinates'], part_data['Velocities'], agn_position)

    # Calculate the average radial velocity
    average_radial_velocity = np.mean(radial_velocities)
    
    average_radial_velocity_list.append(average_radial_velocity)

# Print the average divergence
print("Average radial velocity values:")
print(len(average_radial_velocity_list))

#read the file    
df = pd.read_csv('z=1.csv')
#append them to the new column 
df['Average radial velocity'] = average_radial_velocity_list
df.to_csv('z=1.csv', index=False)

Average radial velocity values:
10000


In [14]:
# List to store Star formation rate
Veldip_values = []

# Loop over each subhalo ID and there snapshot
for id in ids:
    #load the galaxies
    Veldip = il.groupcat.loadSingle(basePath, snapshot, subhaloID=id)
    #extract the value of the velocity
    Veldip_value = sfr['SubhaloVelDisp']
    Veldip_values.append(Veldip_value)
    
print("Velelocity values:")
print(len(Veldip_values))

df = pd.read_csv('z=1.csv')
df['Velocity disp'] = Veldip_values
df.to_csv('z=1.csv', index=False)

Velelocity values:
10000


In [15]:
# List to store BHmdot
bhmdot_values = []

fields = 'BH_Mdot'

# Loop over each subhalo ID and there snapshot
for id in ids:
    # Load the BH data for this snapshot and subhalo ID.
    bh_data = il.snapshot.loadSubhalo(basePath, snapshot, id, "BH", fields=fields)

    # Calculate the BHmdot.
    if isinstance(bh_data, np.ndarray) and bh_data.size > 0:  # Check if bh_data is a non-empty array.
        bh_dot = bh_data[0] * ((1e10 * Msun) / (h * Gyr_to_s)) #covert the bh mases
    elif isinstance(bh_data, dict) and 'BH_Mdot' in bh_data and len(bh_data['BH_Mdot']) > 0:  # Check if there is BH data.
        bh_dot = bh_data['BH_Mdot'][0] * ((1e10 * Msun) / (h * Gyr_to_s))
    else:  # If no BH data, set the BHmdot to zero.
        bh_dot = 0.0
        
    #covert the BHmdot to log10 
    bh_dot = np.log10(bh_dot)

    # Append the BHmdot to the list.
    bhmdot_values.append(bh_dot)
    
    
# Print the BHmdot
print("Bhmdot:")
print(len(bhmdot_values)) 

df = pd.read_csv('z=1.csv')
df['Bhmdot'] = bhmdot_values
df.to_csv('z=1.csv', index=False)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:20: RuntimeWarning: divide by zero encountered in log10


Bhmdot:
10000


In [16]:
# List to store BH masses
bhmass_values = []

fields = 'BH_Mass'

# Loop over each subhalo ID and there snapshot
for id in ids:
    # Load the BH data for this snapshot and subhalo ID.
    bh_data = il.snapshot.loadSubhalo(basePath, snapshot, id, "BH", fields=fields)

    # Calculate the BH mass.
    if isinstance(bh_data, np.ndarray) and bh_data.size > 0:  # Check if bh_data is a non-empty array.
        bh_mass = bh_data[0] * ((1e10) / h) #covert the bh mases
    elif isinstance(bh_data, dict) and 'BH_Mass' in bh_data and len(bh_data['BH_Mass']) > 0:  # Check if there is BH data.
        bh_mass = bh_data['BH_Mass'][0] * ((1e10) / h)
    else:  # If no BH data, set the BH mass to zero.
        bh_mass = 0.0
        
    #covert the bh masess to log10 
    bh_mass = np.log10(bh_mass)

    # Append the BH mass to the list.
    bhmass_values.append(bh_mass)
    
    
# Print the bh masess
print("Blackhole Masses:")
print(len(bhmass_values)) 

df = pd.read_csv('z=1.csv')
df['Blackhole Masses'] = bhmass_values
df.to_csv('z=1.csv', index=False)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:20: RuntimeWarning: divide by zero encountered in log10


Blackhole Masses:
10000


In [17]:
# List to store Metallicity Star formation rate
msfr_values = []

# Loop over each subhalo ID
for id in ids:
    # Load the galaxy data
    msfr = il.groupcat.loadSingle(basePath, snapshot, subhaloID=id)
    # Extract the value of the metallicity SFR
    msfr_value = msfr['SubhaloGasMetallicitySfr']
    msfr_values.append(msfr_value)

print("Metallicity SFR values:")
print(len(msfr_values))

df = pd.read_csv('z=1.csv')
df['MSFR'] = msfr_values
df.to_csv('z=1.csv', index=False)

Metallicity SFR values:
10000


In [18]:
# List to store U-band photometric data
uv_values = []

# Loop over each subhalo ID
for id in ids:
    # Load the galaxy data
    photometrics = il.groupcat.loadSingle(basePath, snapshot, subhaloID=id)
    # Extract the U-band photometric data (which is at index 0)
    uv_band = photometrics['SubhaloStellarPhotometrics'][0]
    uv_values.append(uv_band)
    
print("U-band values:")
print(len(uv_values))

df = pd.read_csv('z=1.csv')
df['UV_Band'] = uv_values
df.to_csv('z=1.csv', index=False)

U-band values:
10000


In [19]:
#lets view the compilation of the file 
df = pd.read_csv('z=1.csv')
print(df.head())

       ID     E_feed      L_bol  SFRinhalfrad  Average divergence  \
0  199097  44.350290  44.350290      0.000000           -0.611396   
1  210307  43.252561  43.252561      0.414761           -0.348660   
2  212195  44.126306  44.126306      0.247022            4.219759   
3  222547  43.732386  43.732386      0.000000            2.245786   
4  223589  43.102978  43.102978      0.293753            3.240826   

   Velocity disp     Bhmdot  Blackhole Masses      MSFR    UV_Band  
0      59.869755  24.095618          8.774169  0.012307 -22.445345  
1      59.869755  22.997890          8.478360  0.012681 -23.001787  
2      59.869755  23.871635          8.609567  0.008637 -22.212519  
3      59.869755  23.477714          8.629783  0.010330 -22.015049  
4      59.869755  22.848307          8.642096  0.009274 -22.436623  


In [ ]:
df['Velocity disp'].unique

<bound method Series.unique of 0       59.869755
1       59.869755
2       59.869755
3       59.869755
4       59.869755
5       59.869755
6       59.869755
7       59.869755
8       59.869755
9       59.869755
10      59.869755
11      59.869755
12      59.869755
13      59.869755
14      59.869755
15      59.869755
16      59.869755
17      59.869755
18      59.869755
19      59.869755
20      59.869755
21      59.869755
22      59.869755
23      59.869755
24      59.869755
25      59.869755
26      59.869755
27      59.869755
28      59.869755
29      59.869755
          ...    
9970    59.869755
9971    59.869755
9972    59.869755
9973    59.869755
9974    59.869755
9975    59.869755
9976    59.869755
9977    59.869755
9978    59.869755
9979    59.869755
9980    59.869755
9981    59.869755
9982    59.869755
9983    59.869755
9984    59.869755
9985    59.869755
9986    59.869755
9987    59.869755
9988    59.869755
9989    59.869755
9990    59.869755
9991    59.869755
9992    59.8697

it's dicided we dont need Velocity dispersion 

In [ ]:
# Drop a column (e.g., "column_to_drop")
df = df.drop('Velocity disp', axis=1)

# Save to CSV file
df.to_csv('z=1.csv', index=False)

### Prediction data for z=0.5

In [3]:
snapshot = 67

#load the index of Subhalo table of the first (central) Subfind subhalo within this FoF group
GroupFirstSub = il.groupcat.loadHalos(basePath,snapshot,fields=['GroupFirstSub'])
print(GroupFirstSub.dtype, GroupFirstSub.shape)


#load the integer counter of the total number of particles/cells, split into the six different types, in this group.
GroupLenType = il.groupcat.loadHalos(basePath,snapshot,fields=['GroupLenType'])
print(GroupLenType.dtype, GroupLenType.shape)

int32 (6572201,)
int32 (6572201, 6)


In [4]:
# Particle type number for black holes
ptNumBH = il.snapshot.partTypeNum('bh')

# Array of number of black holes in each halo
numBH = GroupLenType[:, ptNumBH] #ptNumBH = 5

# Find indices of halos that contain exactly one black hole and at least one subhalo
w = np.where((numBH == 1) & (GroupFirstSub >= 0))[0]

# Find the first subhalo in each of these halos
galaxies = GroupFirstSub[w]

len(galaxies)

22824

In [5]:
#ids of 10000 most massive galaxies
ids = galaxies[0:10000]
len(ids)

10000

In [5]:
# Convert list to pandas DataFrame
df = pd.DataFrame(ids, columns=['ID'])

# Save to CSV
df.to_csv('z=0_5.csv', index=False)

In [ ]:
#lets view the compilation of the file 
df = pd.read_csv('z=0_5.csv')
print(df.head())

       ID
0  277421
1  288670
2  300406
3  303392
4  313327


In [ ]:
# Create empty lists to store the fraction χ of the Eddington accretion rate values
χ_rate_list1 = []  # for χ_rate >= 1
χ_rate_list2 = []  # for χ_rate < 1

fields=["BH_Mass", "BH_Mdot", "BH_MdotEddington"]


# Loop over each subhalo ID
for id in ids:
    # Load black hole properties for specific subhalo
    blackholes = il.snapshot.loadSubhalo(basePath, snapshot, id, "BH", fields=fields)
    
    # Calculate the BHdot.
    if isinstance(blackholes, np.ndarray) and blackholes.size > 0 and blackholes['BH_Mdot'][0] != 0:  
        bhdot = blackholes['BH_Mdot'][0]
    elif isinstance(blackholes, dict) and 'BH_Mdot' in blackholes and len(blackholes['BH_Mdot']) > 0 and blackholes['BH_Mdot'][0] != 0: 
        bhdot = blackholes['BH_Mdot'][0]
    else:  
        bhdot = np.nan  # Set to np.nan so that the resulting χ_rate will be np.nan.
        
    # Calculate the BH eddington.
    if isinstance(blackholes, np.ndarray) and blackholes.size > 0 and blackholes['BH_MdotEddington'][0] != 0:  
        bhedd = blackholes['BH_MdotEddington'][0]
    elif isinstance(blackholes, dict) and 'BH_MdotEddington' in blackholes and len(blackholes['BH_MdotEddington']) > 0 and blackholes['BH_MdotEddington'][0] != 0:  
        bhedd = blackholes['BH_MdotEddington'][0]
    else:  
        bhedd = np.nan  # Set to np.nan so that the resulting χ_rate will be np.nan.

    # Calculate χ_rate
    χ_rate = (bhdot / bhedd)
        
    # Append χ_rate to the appropriate list
    if χ_rate >= 1:
        χ_rate_list1.append(χ_rate)
    else:
        χ_rate_list2.append(χ_rate)

# Print fraction χ values
print("χ values for χ_rate >= 1:")
print(len(χ_rate_list1))

print("χ values for χ_rate < 1:")
print(len(χ_rate_list2))

χ values for χ_rate >= 1:
1
χ values for χ_rate < 1:
9999


In [11]:
# Create empty list to store of fraction χ of the Eddington accretion rate values
λ_rate_list = []

fields=["BH_Mass", "BH_Mdot", "BH_MdotEddington"]

# Loop over each subhalo ID and there snapshot
for id in ids:
    # Load black hole properties for specific subhalo
    blackholes = il.snapshot.loadSubhalo(basePath, snapshot, id, "BH", fields=fields)
    
    # Calculate the BHdot.
    if isinstance(blackholes, np.ndarray) and blackholes.size > 0 and blackholes['BH_Mdot'][0] != 0:  
        bhdot = blackholes['BH_Mdot'][0]
    elif isinstance(blackholes, dict) and 'BH_Mdot' in blackholes and len(blackholes['BH_Mdot']) > 0 and blackholes['BH_Mdot'][0] != 0: 
        bhdot = blackholes['BH_Mdot'][0]
    else:  
        bhdot = np.nan  # Set to np.nan so that the resulting λ_rate will be np.nan.
        
    # Calculate the BH eddington.
    if isinstance(blackholes, np.ndarray) and blackholes.size > 0 and blackholes['BH_MdotEddington'][0] != 0:  
        bhedd = blackholes['BH_MdotEddington'][0]
    elif isinstance(blackholes, dict) and 'BH_MdotEddington' in blackholes and len(blackholes['BH_MdotEddington']) > 0 and blackholes['BH_MdotEddington'][0] != 0:  
        bhedd = blackholes['BH_MdotEddington'][0]
    else:  
        bhedd = np.nan  # Set to np.nan so that the resulting λ_rate will be np.nan.

        
    # Calculate λ_rate
    λ_rate = (bhdot / bhedd)

    
    # Append λ_rate to the list
    λ_rate_list.append(λ_rate)


print("λ values:")
print(len(λ_rate_list))

df = pd.read_csv('z=0_5.csv')
df['λ'] = λ_rate_list
df.to_csv('z=0_5.csv', index=False)

λ values:
10000


In [ ]:
# Create empty list to store the feedback energy of the AGN values
E_feed_list = []


Msun = 1.989e33 # in grams
Gyr_to_s = 3.15576e16 # in seconds
h = 0.6774

fields=["BH_Mass", "BH_Mdot"]

# Loop over each subhalo ID and there snapshot
for id in ids:
    # Load black hole properties for specific subhalo
    blackholes = il.snapshot.loadSubhalo(basePath, snapshot, id, "BH", fields=fields)
    
    # Calculate the BHdot.
    if isinstance(blackholes, np.ndarray) and blackholes.size > 0 and blackholes['BH_Mdot'][0] != 0:  
        bhdot = blackholes['BH_Mdot'][0] * ((1e10 * Msun) / (h * Gyr_to_s))
    elif isinstance(blackholes, dict) and 'BH_Mdot' in blackholes and len(blackholes['BH_Mdot']) > 0 and blackholes['BH_Mdot'][0] != 0: 
        bhdot = blackholes['BH_Mdot'][0] * ((1e10 * Msun) / (h * Gyr_to_s))
    else:  
        bhdot = np.nan  # Set to np.nan so that the resulting feedback energy will be np.nan.

        
    # Calculate feedback energy
    E_feed = e_fm * bhdot * c**2

    
    # Append feedback energy to the list
    E_feed_list.append(np.log10(E_feed))


# Print feedback energy values
print("E_feed values:")

print(len(E_feed_list))

df = pd.read_csv('z=0_5.csv')
df['E_feed'] = E_feed_list
df.to_csv('z=0_5.csv', index=False)

E_feed values:
10000


In [ ]:
# Create empty list to store the bolometric luminosity values
L_list = []

fields=["BH_Mass", "BH_Mdot"]

# Loop over each subhalo ID and there snapshot
for id in ids:
    # Load black hole properties for specific subhalo
    blackholes = il.snapshot.loadSubhalo(basePath, snapshot, id, "BH", fields=fields)
    
    # Calculate the BHdot.
    if isinstance(blackholes, np.ndarray) and blackholes.size > 0 and blackholes['BH_Mdot'][0] != 0:  
        bhdot = blackholes['BH_Mdot'][0] * ((1e10 * Msun) / (h * Gyr_to_s))
    elif isinstance(blackholes, dict) and 'BH_Mdot' in blackholes and len(blackholes['BH_Mdot']) > 0 and blackholes['BH_Mdot'][0] != 0: 
        bhdot = blackholes['BH_Mdot'][0] * ((1e10 * Msun) / (h * Gyr_to_s))
    else:  
        bhdot = np.nan  # Set to np.nan so that the resulting L_bol will be np.nan.

    # Calculate L_bol
    L_bol = e_r * bhdot * c**2

    
    # Append bolometric luminosity to the list
    L_list.append(np.log10(L_bol))


# Print bolometric luminosity values
print("Bol Luminosity values:")
print(len(L_list))

df = pd.read_csv('z=0_5.csv')
df['L_bol'] = L_list
df.to_csv('z=0_5.csv', index=False)

Bol Luminosity values:
10000


In [ ]:
# List to store Star formation rate
sfr_values = []

# Loop over each subhalo ID and there snapshot
for id in ids:
    #load the galaxies
    sfr = il.groupcat.loadSingle(basePath, snapshot, subhaloID=id)
    #extract the value of the sfr
    sfr_value = sfr['SubhaloSFRinHalfRad']
    sfr_values.append(sfr_value)
    
print("SFR values:")
print(len(sfr_values))

df = pd.read_csv('z=0_5.csv')
df['SFRinhalfrad'] = sfr_values
df.to_csv('z=0_5.csv', index=False)

SFR values:
10000


In [ ]:
def calculate_divergence(part_data, agn_position):
    # Check if 'Coordinates' is in the loaded data
    if 'Coordinates' not in part_data:
        return np.nan

    # Calculate the position of the particles relative to the AGN
    relative_pos = part_data['Coordinates'] - agn_position

    # Calculate the position boundaries
    padding = 10  # This value can be adjusted as needed
    min_x, min_y, min_z = np.min(relative_pos, axis=0) - padding
    max_x, max_y, max_z = np.max(relative_pos, axis=0) + padding

    # Create a grid
    grid_x, grid_y, grid_z = np.mgrid[min_x:max_x:100j, min_y:max_y:100j, min_z:max_z:100j]

    # Interpolate the velocities onto the grid
    grid_vx = interpolate.griddata(relative_pos, part_data['Velocities'][:, 0], (grid_x, grid_y, grid_z), method='nearest')
    grid_vy = interpolate.griddata(relative_pos, part_data['Velocities'][:, 1], (grid_x, grid_y, grid_z), method='nearest')
    grid_vz = interpolate.griddata(relative_pos, part_data['Velocities'][:, 2], (grid_x, grid_y, grid_z), method='nearest')

    # Calculate the divergence
    divergence = np.gradient(grid_vx, axis=0) + np.gradient(grid_vy, axis=1) + np.gradient(grid_vz, axis=2)
    
    # Return the average divergence    
    return np.mean(divergence)

# Define lists to store the results
avg_divergence_list = []

# Loop over each subhalo ID and there snapshot
for id in ids:
    # Load the particle data for the subhalo
    part_data = il.snapshot.loadSubhalo(basePath, snapshot, id, partType='gas', fields=['Coordinates', 'Velocities'])
    
    agn_position = il.groupcat.loadSingle(basePath, snapshot, subhaloID=id)
    agn_position = agn_position['SubhaloPos']

    # Calculate the divergence
    average_divergence = calculate_divergence(part_data, agn_position)

    # Store the results
    avg_divergence_list.append(average_divergence)

# Print the average divergence
print("Average divergence velocity values:")
print(len(avg_divergence_list))

#read the file    
df = pd.read_csv('z=0_5.csv')
#append them to the new column 
df['Average divergence'] = avg_divergence_list
df.to_csv('z=0_5.csv', index=False)

In [6]:
def calculate_radial_velocity(positions, velocities, agn_position):
    # Calculate the relative positions and distances of the particles
    relative_positions = positions - agn_position
    distances = np.linalg.norm(relative_positions, axis=1)

    # Calculate the radial velocities
    radial_velocities = np.sum(relative_positions * velocities, axis=1) / distances

    return radial_velocities

# Define lists to store the results
average_radial_velocity_list = []


# Loop over each subhalo ID and there snapshot
for id in ids:
    # Get the main halo ID (Group Number) for the subhalo
    main_halo_id = il.groupcat.loadSingle(basePath, snapshot, subhaloID=id)['SubhaloGrNr']
    
    # Load the properties of the main halo
    main_halo_properties = il.groupcat.loadSingle(basePath, snapshot, haloID=main_halo_id)
    
    # Get the position of the main halo
    agn_position = main_halo_properties['GroupPos']
    
    # Load the particle data for the halo
    part_data = il.snapshot.loadHalo(basePath, snapshot, main_halo_id, partType='gas', fields=['Coordinates', 'Velocities'])

    # Calculate the radial velocities
    radial_velocities = calculate_radial_velocity(part_data['Coordinates'], part_data['Velocities'], agn_position)

    # Calculate the average radial velocity
    average_radial_velocity = np.mean(radial_velocities)
    
    average_radial_velocity_list.append(average_radial_velocity)

# Print the average divergence
print("Average radial velocity values:")
print(len(average_radial_velocity_list))

#read the file    
df = pd.read_csv('z=0_5.csv')
#append them to the new column 
df['Average radial velocity'] = average_radial_velocity_list
df.to_csv('z=0_5.csv', index=False)

Average radial velocity values:
10000


In [ ]:
# List to store BH masses
bhmdot_values = []

fields = 'BH_Mdot'

# Loop over each subhalo ID and there snapshot
for id in ids:
    # Load the BH data for this snapshot and subhalo ID.
    bh_data = il.snapshot.loadSubhalo(basePath, snapshot, id, "BH", fields=fields)

    # Calculate the BH mass.
    if isinstance(bh_data, np.ndarray) and bh_data.size > 0:  # Check if bh_data is a non-empty array.
        bh_dot = bh_data[0] * ((1e10 * Msun) / (h * Gyr_to_s)) #covert the bh mases
    elif isinstance(bh_data, dict) and 'BH_Mdot' in bh_data and len(bh_data['BH_Mdot']) > 0:  # Check if there is BH data.
        bh_dot = bh_data['BH_Mdot'][0] * ((1e10 * Msun) / (h * Gyr_to_s))
    else:  # If no BH data, set the BH mass to zero.
        bh_dot = 0.0
        
    #covert the bh masess to log10 
    bh_dot = np.log10(bh_dot)

    # Append the BH mass to the list.
    bhmdot_values.append(bh_dot)
    
    
# Print the average divergence
print("Bhmdot:")
print(len(bhmdot_values)) 

df = pd.read_csv('z=0_5.csv')
df['Bhmdot'] = bhmdot_values
df.to_csv('z=0_5.csv', index=False)

In [ ]:
# List to store BH masses
bhmass_values = []

fields = 'BH_Mass'

# Loop over each subhalo ID and there snapshot
for id in ids:
    # Load the BH data for this snapshot and subhalo ID.
    bh_data = il.snapshot.loadSubhalo(basePath, snapshot, id, "BH", fields=fields)

    # Calculate the BH mass.
    if isinstance(bh_data, np.ndarray) and bh_data.size > 0:  # Check if bh_data is a non-empty array.
        bh_mass = bh_data[0] * ((1e10) / h) #covert the bh mases
    elif isinstance(bh_data, dict) and 'BH_Mass' in bh_data and len(bh_data['BH_Mass']) > 0:  # Check if there is BH data.
        bh_mass = bh_data['BH_Mass'][0] * ((1e10) / h)
    else:  # If no BH data, set the BH mass to zero.
        bh_mass = 0.0
        
    #covert the bh masess to log10 
    bh_mass = np.log10(bh_mass)

    # Append the BH mass to the list.
    bhmass_values.append(bh_mass)
    
    
# Print the average divergence
print("Blackhole Masses:")
print(len(bhmass_values)) 

df = pd.read_csv('z=0_5.csv')
df['Blackhole Masses'] = bhmass_values
df.to_csv('z=0_5.csv', index=False)

In [ ]:
# List to store metallicity Star formation rate
msfr_values = []

# Loop over each subhalo ID
for id in ids:
    # Load the galaxy data
    msfr = il.groupcat.loadSingle(basePath, snapshot, subhaloID=id)
    # Extract the value of the metallicity SFR
    msfr_value = msfr['SubhaloGasMetallicitySfr']
    msfr_values.append(msfr_value)

print("Metallicity SFR values:")
print(len(msfr_values))

df = pd.read_csv('z=0_5.csv')
df['MSFR'] = msfr_values
df.to_csv('z=0_5.csv', index=False)

In [ ]:
# List to store U-band photometric data
uv_values = []

# Loop over each subhalo ID
for id in ids:
    # Load the galaxy data
    photometrics = il.groupcat.loadSingle(basePath, snapshot, subhaloID=id)
    # Extract the U-band photometric data (which is at index 0)
    uv_band = photometrics['SubhaloStellarPhotometrics'][0]
    uv_values.append(uv_band)
    
print("U-band values:")
print(len(uv_values))

df = pd.read_csv('z=0_5.csv')
df['UV_Band'] = uv_values
df.to_csv('z=0_5.csv', index=False)

In [ ]:
#lets view the compilation of the file 
df = pd.read_csv('z=0_5.csv')
print(df.head())

### Prediction data for z=0

In [7]:
snapshot = 99

#load the index of Subhalo table of the first (central) Subfind subhalo within this FoF group
GroupFirstSub = il.groupcat.loadHalos(basePath,snapshot,fields=['GroupFirstSub'])
print(GroupFirstSub.dtype, GroupFirstSub.shape)


#load the integer counter of the total number of particles/cells, split into the six different types, in this group.
GroupLenType = il.groupcat.loadHalos(basePath,snapshot,fields=['GroupLenType'])
print(GroupLenType.dtype, GroupLenType.shape)

int32 (6291349,)
int32 (6291349, 6)


In [8]:
# Particle type number for black holes
ptNumBH = il.snapshot.partTypeNum('bh')

# Array of number of black holes in each halo
numBH = GroupLenType[:, ptNumBH] #ptNumBH = 5

# Find indices of halos that contain exactly one black hole and at least one subhalo
w = np.where((numBH == 1) & (GroupFirstSub >= 0))[0]

# Find the first subhalo in each of these halos
galaxies = GroupFirstSub[w]

len(galaxies)

22671

In [9]:
#ids of 10000 most massive galaxies
ids = galaxies[0:10000]
len(ids)

10000

In [5]:
# Convert list to pandas DataFrame
df = pd.DataFrame(ids, columns=['ID'])

# Save to CSV
df.to_csv('z=0.csv', index=False)

In [6]:
#lets view the compilation of the file 
df = pd.read_csv('z=0.csv')
print(df.head())

       ID
0  356678
1  359811
2  377398
3  381608
4  384914


In [7]:
# Create empty lists to store the fraction χ of the Eddington accretion rate values
χ_rate_list1 = []  # for χ_rate >= 1
χ_rate_list2 = []  # for χ_rate < 1

fields=["BH_Mass", "BH_Mdot", "BH_MdotEddington"]


# Loop over each subhalo ID
for id in ids:
    # Load black hole properties for specific subhalo
    blackholes = il.snapshot.loadSubhalo(basePath, snapshot, id, "BH", fields=fields)
    
    # Calculate the BHdot.
    if isinstance(blackholes, np.ndarray) and blackholes.size > 0 and blackholes['BH_Mdot'][0] != 0:  
        bhdot = blackholes['BH_Mdot'][0]
    elif isinstance(blackholes, dict) and 'BH_Mdot' in blackholes and len(blackholes['BH_Mdot']) > 0 and blackholes['BH_Mdot'][0] != 0: 
        bhdot = blackholes['BH_Mdot'][0]
    else:  
        bhdot = np.nan  # Set to np.nan so that the resulting χ_rate will be np.nan.
        
    # Calculate the BH eddington.
    if isinstance(blackholes, np.ndarray) and blackholes.size > 0 and blackholes['BH_MdotEddington'][0] != 0:  
        bhedd = blackholes['BH_MdotEddington'][0]
    elif isinstance(blackholes, dict) and 'BH_MdotEddington' in blackholes and len(blackholes['BH_MdotEddington']) > 0 and blackholes['BH_MdotEddington'][0] != 0:  
        bhedd = blackholes['BH_MdotEddington'][0]
    else:  
        bhedd = np.nan  # Set to np.nan so that the resulting χ_rate will be np.nan.

    # Calculate χ_rate
    χ_rate = (bhdot / bhedd)
    
    # Append χ_rate to the appropriate list
    if χ_rate >= 1:
        χ_rate_list1.append(χ_rate)
    else:
        χ_rate_list2.append(χ_rate)

# Print fraction χ values
print("χ values for χ_rate >= 1:")
print(len(χ_rate_list1))

print("χ values for χ_rate < 1:")
print(len(χ_rate_list2))

χ values for χ_rate >= 1:
0
χ values for χ_rate < 1:
10000


In [15]:
# Create empty list to store of fraction χ of the Eddington accretion rate values
λ_rate_list = []

fields=["BH_Mass", "BH_Mdot", "BH_MdotEddington"]

# Loop over each subhalo ID and there snapshot
for id in ids:
    # Load black hole properties for specific subhalo
    blackholes = il.snapshot.loadSubhalo(basePath, snapshot, id, "BH", fields=fields)
    
    # Calculate the BHdot.
    if isinstance(blackholes, np.ndarray) and blackholes.size > 0 and blackholes['BH_Mdot'][0] != 0:  
        bhdot = blackholes['BH_Mdot'][0]
    elif isinstance(blackholes, dict) and 'BH_Mdot' in blackholes and len(blackholes['BH_Mdot']) > 0 and blackholes['BH_Mdot'][0] != 0: 
        bhdot = blackholes['BH_Mdot'][0]
    else:  
        bhdot = np.nan  # Set to np.nan so that the resulting λ_rate will be np.nan.
        
    # Calculate the BH eddington.
    if isinstance(blackholes, np.ndarray) and blackholes.size > 0 and blackholes['BH_MdotEddington'][0] != 0:  
        bhedd = blackholes['BH_MdotEddington'][0]
    elif isinstance(blackholes, dict) and 'BH_MdotEddington' in blackholes and len(blackholes['BH_MdotEddington']) > 0 and blackholes['BH_MdotEddington'][0] != 0:  
        bhedd = blackholes['BH_MdotEddington'][0]
    else:  
        bhedd = np.nan  # Set to np.nan so that the resulting λ_rate will be np.nan.

        
    # Calculate λ_rate
    λ_rate = (bhdot / bhedd)

    
    # Append λ_rate to the list
    λ_rate_list.append(λ_rate)


print("λ values:")
print(len(λ_rate_list))

df = pd.read_csv('z=0.csv')
df['λ'] = λ_rate_list
df.to_csv('z=0.csv', index=False)

λ values:
10000


In [8]:
# Create empty list to store the feedback energy of the AGN values
E_feed_list = []


Msun = 1.989e33 # in grams
Gyr_to_s = 3.15576e16 # in seconds
h = 0.6774

fields=["BH_Mass", "BH_Mdot"]

# Loop over each subhalo ID and there snapshot
for id in ids:
    # Load black hole properties for specific subhalo
    blackholes = il.snapshot.loadSubhalo(basePath, snapshot, id, "BH", fields=fields)
    
    # Calculate the BHdot.
    if isinstance(blackholes, np.ndarray) and blackholes.size > 0 and blackholes['BH_Mdot'][0] != 0:  
        bhdot = blackholes['BH_Mdot'][0] * ((1e10 * Msun) / (h * Gyr_to_s))
    elif isinstance(blackholes, dict) and 'BH_Mdot' in blackholes and len(blackholes['BH_Mdot']) > 0 and blackholes['BH_Mdot'][0] != 0: 
        bhdot = blackholes['BH_Mdot'][0] * ((1e10 * Msun) / (h * Gyr_to_s))
    else:  
        bhdot = np.nan  # Set to np.nan so that the resulting feedback energy will be np.nan.

        
    # Calculate feedback energy
    E_feed = e_fm * bhdot * c**2

    
    # Append feedback energy to the list
    E_feed_list.append(np.log10(E_feed))


# Print feedback energy values
print("E_feed values:")

print(len(E_feed_list))

df = pd.read_csv('z=0.csv')
df['E_feed'] = E_feed_list
df.to_csv('z=0.csv', index=False)

E_feed values:
10000


In [9]:
# Create empty list to store the bolometric luminosity values
L_list = []

fields=["BH_Mass", "BH_Mdot"]

# Loop over each subhalo ID and there snapshot
for id in ids:
    # Load black hole properties for specific subhalo
    blackholes = il.snapshot.loadSubhalo(basePath, snapshot, id, "BH", fields=fields)
    
    # Calculate the BHdot.
    if isinstance(blackholes, np.ndarray) and blackholes.size > 0 and blackholes['BH_Mdot'][0] != 0:  
        bhdot = blackholes['BH_Mdot'][0] * ((1e10 * Msun) / (h * Gyr_to_s))
    elif isinstance(blackholes, dict) and 'BH_Mdot' in blackholes and len(blackholes['BH_Mdot']) > 0 and blackholes['BH_Mdot'][0] != 0: 
        bhdot = blackholes['BH_Mdot'][0] * ((1e10 * Msun) / (h * Gyr_to_s))
    else:  
        bhdot = np.nan  # Set to np.nan so that the resulting L_bol will be np.nan.

    # Calculate L_bol
    L_bol = e_r * bhdot * c**2

    
    # Append bolometric luminosity to the list
    L_list.append(np.log10(L_bol))


# Print bolometric luminosity values
print("Bol Luminosity values:")
print(len(L_list))

df = pd.read_csv('z=0.csv')
df['L_bol'] = L_list
df.to_csv('z=0.csv', index=False)

Bol Luminosity values:
10000


In [10]:
# List to store Star formation rate
sfr_values = []

# Loop over each subhalo ID and there snapshot
for id in ids:
    #load the galaxies
    sfr = il.groupcat.loadSingle(basePath, snapshot, subhaloID=id)
    #extract the value of the sfr
    sfr_value = sfr['SubhaloSFRinHalfRad']
    sfr_values.append(sfr_value)
    
print("SFR values:")
print(len(sfr_values))

df = pd.read_csv('z=0.csv')
df['SFRinhalfrad'] = sfr_values
df.to_csv('z=0.csv', index=False)

SFR values:
10000


In [ ]:
def calculate_divergence(part_data, agn_position):
    # Check if 'Coordinates' is in the loaded data
    if 'Coordinates' not in part_data:
        return np.nan

    # Calculate the position of the particles relative to the AGN
    relative_pos = part_data['Coordinates'] - agn_position

    # Calculate the position boundaries
    padding = 10  # This value can be adjusted as needed
    min_x, min_y, min_z = np.min(relative_pos, axis=0) - padding
    max_x, max_y, max_z = np.max(relative_pos, axis=0) + padding

    # Create a grid
    grid_x, grid_y, grid_z = np.mgrid[min_x:max_x:100j, min_y:max_y:100j, min_z:max_z:100j]

    # Interpolate the velocities onto the grid
    grid_vx = interpolate.griddata(relative_pos, part_data['Velocities'][:, 0], (grid_x, grid_y, grid_z), method='nearest')
    grid_vy = interpolate.griddata(relative_pos, part_data['Velocities'][:, 1], (grid_x, grid_y, grid_z), method='nearest')
    grid_vz = interpolate.griddata(relative_pos, part_data['Velocities'][:, 2], (grid_x, grid_y, grid_z), method='nearest')

    # Calculate the divergence
    divergence = np.gradient(grid_vx, axis=0) + np.gradient(grid_vy, axis=1) + np.gradient(grid_vz, axis=2)
    
    # Return the average divergence    
    return np.mean(divergence)

# Define lists to store the results
avg_divergence_list = []

# Loop over each subhalo ID and there snapshot
for id in ids:
    # Load the particle data for the subhalo
    part_data = il.snapshot.loadSubhalo(basePath, snapshot, id, partType='gas', fields=['Coordinates', 'Velocities'])
    
    agn_position = il.groupcat.loadSingle(basePath, snapshot, subhaloID=id)
    agn_position = agn_position['SubhaloPos']

    # Calculate the divergence
    average_divergence = calculate_divergence(part_data, agn_position)

    # Store the results
    avg_divergence_list.append(average_divergence)

# Print the average divergence
print("Average divergence velocity values:")
print(avg_divergence_list)

#read the file    
df = pd.read_csv('z=0.csv')
#append them to the new column 
df['Average divergence'] = avg_divergence_list
df.to_csv('z=0.csv', index=False)

In [10]:
def calculate_radial_velocity(positions, velocities, agn_position):
    # Calculate the relative positions and distances of the particles
    relative_positions = positions - agn_position
    distances = np.linalg.norm(relative_positions, axis=1)

    # Calculate the radial velocities
    radial_velocities = np.sum(relative_positions * velocities, axis=1) / distances

    return radial_velocities

# Define lists to store the results
average_radial_velocity_list = []


# Loop over each subhalo ID and there snapshot
for id in ids:
    # Get the main halo ID (Group Number) for the subhalo
    main_halo_id = il.groupcat.loadSingle(basePath, snapshot, subhaloID=id)['SubhaloGrNr']
    
    # Load the properties of the main halo
    main_halo_properties = il.groupcat.loadSingle(basePath, snapshot, haloID=main_halo_id)
    
    # Get the position of the main halo
    agn_position = main_halo_properties['GroupPos']
    
    # Load the particle data for the halo
    part_data = il.snapshot.loadHalo(basePath, snapshot, main_halo_id, partType='gas', fields=['Coordinates', 'Velocities'])

    # Calculate the radial velocities
    radial_velocities = calculate_radial_velocity(part_data['Coordinates'], part_data['Velocities'], agn_position)

    # Calculate the average radial velocity
    average_radial_velocity = np.mean(radial_velocities)
    
    average_radial_velocity_list.append(average_radial_velocity)

# Print the average divergence
print("Average radial velocity values:")
print(len(average_radial_velocity_list))

#read the file    
df = pd.read_csv('z=0.csv')
#append them to the new column 
df['Average radial velocity'] = average_radial_velocity_list
df.to_csv('z=0.csv', index=False)

Average radial velocity values:
10000


In [14]:
# List to store BH masses
bhmdot_values = []

fields = 'BH_Mdot'

# Loop over each subhalo ID and there snapshot
for id in ids:
    # Load the BH data for this snapshot and subhalo ID.
    bh_data = il.snapshot.loadSubhalo(basePath, snapshot, id, "BH", fields=fields)

    # Calculate the BH mass.
    if isinstance(bh_data, np.ndarray) and bh_data.size > 0:  # Check if bh_data is a non-empty array.
        bh_dot = bh_data[0] * ((1e10 * Msun) / (h * Gyr_to_s)) #covert the bh mases
    elif isinstance(bh_data, dict) and 'BH_Mdot' in bh_data and len(bh_data['BH_Mdot']) > 0:  # Check if there is BH data.
        bh_dot = bh_data['BH_Mdot'][0] * ((1e10 * Msun) / (h * Gyr_to_s))
    else:  # If no BH data, set the BH mass to zero.
        bh_dot = 0.0
        
    #covert the bh masess to log10 
    bh_dot = np.log10(bh_dot)

    # Append the BH mass to the list.
    bhmdot_values.append(bh_dot)
    
    
# Print the average divergence
print("Bhmdot:")
print(len(bhmdot_values)) 

df = pd.read_csv('z=0.csv')
df['Bhmdot'] = bhmdot_values
df.to_csv('z=0.csv', index=False)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:20: RuntimeWarning: divide by zero encountered in log10


Bhmdot:
10000


In [15]:
# List to store BH masses
bhmass_values = []

fields = 'BH_Mass'

# Loop over each subhalo ID and there snapshot
for id in ids:
    # Load the BH data for this snapshot and subhalo ID.
    bh_data = il.snapshot.loadSubhalo(basePath, snapshot, id, "BH", fields=fields)

    # Calculate the BH mass.
    if isinstance(bh_data, np.ndarray) and bh_data.size > 0:  # Check if bh_data is a non-empty array.
        bh_mass = bh_data[0] * ((1e10) / h) #covert the bh mases
    elif isinstance(bh_data, dict) and 'BH_Mass' in bh_data and len(bh_data['BH_Mass']) > 0:  # Check if there is BH data.
        bh_mass = bh_data['BH_Mass'][0] * ((1e10) / h)
    else:  # If no BH data, set the BH mass to zero.
        bh_mass = 0.0
        
    #covert the bh masess to log10 
    bh_mass = np.log10(bh_mass)

    # Append the BH mass to the list.
    bhmass_values.append(bh_mass)
    
    
# Print the average divergence
print("Blackhole Masses:")
print(len(bhmass_values)) 

df = pd.read_csv('z=0.csv')
df['Blackhole Masses'] = bhmass_values
df.to_csv('z=0.csv', index=False)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:20: RuntimeWarning: divide by zero encountered in log10


Blackhole Masses:
10000


In [16]:
# List to store Metallicity Star formation rate
msfr_values = []

# Loop over each subhalo ID
for id in ids:
    # Load the galaxy data
    msfr = il.groupcat.loadSingle(basePath, snapshot, subhaloID=id)
    # Extract the value of the metallicity SFR
    msfr_value = msfr['SubhaloGasMetallicitySfr']
    msfr_values.append(msfr_value)

print("Metallicity SFR values:")
print(len(msfr_values))

df = pd.read_csv('z=0.csv')
df['MSFR'] = msfr_values
df.to_csv('z=0.csv', index=False)

Metallicity SFR values:
10000


In [17]:
# List to store U-band photometric data
uv_values = []

# Loop over each subhalo ID
for id in ids:
    # Load the galaxy data
    photometrics = il.groupcat.loadSingle(basePath, snapshot, subhaloID=id)
    # Extract the U-band photometric data (which is at index 0)
    uv_band = photometrics['SubhaloStellarPhotometrics'][0]
    uv_values.append(uv_band)
    
print("U-band values:")
print(len(uv_values))

df = pd.read_csv('z=0.csv')
df['UV_Band'] = uv_values
df.to_csv('z=0.csv', index=False)

U-band values:
10000


In [18]:
#lets view the compilation of the file 
df = pd.read_csv('z=0.csv')
print(df.head())

       ID     E_feed      L_bol  SFRinhalfrad  Average divergence     Bhmdot  \
0  356678  43.172570  43.172570           0.0           -0.933217  22.917899   
1  359811  42.544454  42.544454           0.0           -1.404874  22.289783   
2  377398  42.564474  42.564474           0.0            0.805930  22.309802   
3  381608  41.487624  41.487624           0.0            0.321204  21.232953   
4  384914  42.692935  42.692935           0.0           -0.319989  22.438263   

   Blackhole Masses      MSFR    UV_Band  
0          8.905789  0.000000 -21.000128  
1          9.003120  0.005017 -20.239038  
2          8.776723  0.013295 -20.981514  
3          8.836200  0.000000 -20.463696  
4          8.741881  0.005737 -20.651108  


----------

Here is the code for getting the Average Divergence velocity of gas in Halo, though due to immense time processing of more than 12 hours, it was left out of the Data and also because of the decontamination of the Halo.

In [ ]:
def calculate_divergence(part_data, agn_position):
    # Calculate the position of the particles relative to the AGN
    relative_pos = part_data['Coordinates'] - agn_position

    # Calculate the position boundaries
    padding = 1  # This value can be adjusted as needed
    min_x, min_y, min_z = np.min(relative_pos, axis=0) - padding
    max_x, max_y, max_z = np.max(relative_pos, axis=0) + padding

    # Create a grid
    grid_x, grid_y, grid_z = np.mgrid[min_x:max_x:100j, min_y:max_y:100j, min_z:max_z:100j]

    # Interpolate the velocities onto the grid
    grid_vx = interpolate.griddata(relative_pos, part_data['Velocities'][:, 0], (grid_x, grid_y, grid_z), method='nearest')
    grid_vy = interpolate.griddata(relative_pos, part_data['Velocities'][:, 1], (grid_x, grid_y, grid_z), method='nearest')
    grid_vz = interpolate.griddata(relative_pos, part_data['Velocities'][:, 2], (grid_x, grid_y, grid_z), method='nearest')

    # Calculate the divergence
    divergence = np.gradient(grid_vx, axis=0) + np.gradient(grid_vy, axis=1) + np.gradient(grid_vz, axis=2)
    
    # Return the average divergence    
    return np.mean(divergence)

# Define lists to store the results
avg_divergence_list = []

# Loop over each subhalo ID
for id in ids:
    # Get the main halo ID (Group Number) for the subhalo
    main_halo_id = il.groupcat.loadSingle(basePath, snapshot, subhaloID=id)['SubhaloGrNr']
    
    # Load the properties of the main halo
    main_halo_properties = il.groupcat.loadSingle(basePath, snapshot, haloID=main_halo_id)
    
    # Get the position of the main halo
    agn_position = main_halo_properties['GroupPos']
    
    # Load the particle data for the halo
    part_data = il.snapshot.loadHalo(basePath, snapshot, main_halo_id, partType='gas', fields=['Coordinates', 'Velocities'])

    # Calculate the divergence
    average_divergence = calculate_divergence(part_data, agn_position)

    # Store the results
    avg_divergence_list.append(average_divergence)

# Print the average divergence
print("Average divergence velocity values:")
print(len(avg_divergence_list))